In [ ]:
import numpy as np
import os
import pickle
import torch

from Models import generateSamples_training_ErdosRenyi, run, build
from Evaluations import evaluate

model_num = 1
training_switch = -1
k_max = 4
cuda = 0

if model_num == 1:
    model_name = '128 64 32 16'
    hidden_widths = [128,64,32,16]
elif model_num == 2:
    model_name = '64 32 16 8'
    hidden_widths = [64,32,16,8]
elif model_num == 3:
    model_name = '32 16 8 4'
    hidden_widths = [32,16,8,4]
elif model_num == 4:
    model_name = '128 64 32'
    hidden_widths = [128,64]
elif model_num == 5:
    model_name = '64 32 16'
    hidden_widths = [64,32,16]
elif model_num == 6:
    model_name = '32 16 8'
    hidden_widths = [32,16,8]
elif model_num == 7:
    model_name = '128 64'
    hidden_widths = [128,64]
elif model_num == 8:
    model_name = '64 32'
    hidden_widths = [64,32]
elif model_num == 9:
    model_name = '32 16'
    hidden_widths = [32,16]
elif model_num == 0:
    model_name = 'test'
    hidden_widths = [16]
else:
    raise AssertionError('Model not specified.')

if model_num == 1:
    GNN_only = False
else:
    GNN_only = True

if training_switch == -1:
    output_dir = 'unmasked'
elif training_switch == 0:
    output_dir = 'switch0'
elif training_switch == 1:
    output_dir = 'switch1'
elif training_switch == 2:
    output_dir = 'switch2'
elif training_switch == 3:
    output_dir = 'switch3'
elif training_switch == 4:
    output_dir = 'switch4'
elif training_switch == 5:
    output_dir = 'switch5'
elif training_switch == 6:
    output_dir = 'switch6'
elif training_switch == 7:
    output_dir = 'switch7'
elif training_switch == 8:
    output_dir = 'switch8'
elif training_switch == 9:
    output_dir = 'switch9'
elif training_switch == 10:
    output_dir = 'switch10'
elif training_switch == 11:
    output_dir = 'switch11'
elif training_switch == 12:
    output_dir = 'switch12'
elif training_switch == 13:
    output_dir = 'switch13'
elif training_switch == 14:
    output_dir = 'switch14'
else:
    raise AssertionError('Invalid training switch.')

graph_type = 'ErdosRenyi'
sample_dir = os.path.dirname(os.getcwd())+'/samples/'+graph_type+'_lambda_div_size'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

all_out_dir = sample_dir.replace('samples','outputs')
if not os.path.exists(all_out_dir):
    os.makedirs(all_out_dir)
output_dir = all_out_dir + '/' + output_dir

model_dir = output_dir + '/' + model_name
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

seed_metrics = ['random','degree','closeness','betweenness','harmonic','laplacian','pagerank']

In [ ]:
mode = 'random'
for n in [1600]:
    for lbd in [6,5,4,3]:
        
        title = 'n = '+str(n)+', lambda = '+str(lbd)
        print(title)
        dir = model_dir+'/lbd '+str(lbd)
        if not os.path.exists(dir):
            os.makedirs(dir)

        if os.path.exists(model_dir+'/status_'+title+'.pkl'):
            with open(model_dir+'/status_'+title+'.pkl', 'rb') as file:
                status = pickle.load(file)
            if status[0] == True and status[1] == True and status[2] == True and status[3] == True:
                done = True
            else:
                done = False
        else:
            done = False

        if not done:

            p = sample_dir+'/'+graph_type+'_training_'+str(n)+'_'+str(lbd)+'.pkl'
            if os.path.exists(p):
                with open(p, 'rb') as file:
                    samples = pickle.load(file)
            else:
                samples = generateSamples_training_ErdosRenyi(sample_dir,title,200,50,50,n,lbd)
                with open(p, 'wb') as file:
                    pickle.dump(samples, file)

            model1 = run(model_dir,dir,title,samples,hidden_widths,'GCN','mse','adam','cyclic-cosine','reLU',1000,100,training_switch=training_switch,save_model=True,which_cuda=cuda)
            model2 = run(model_dir,dir,title,samples,hidden_widths,'SAGE','mse','adam','cyclic-cosine','reLU',1000,100,training_switch=training_switch,save_model=True,which_cuda=cuda)
            model3 = run(model_dir,dir,title,samples,hidden_widths,'GAT','mse','adam','cyclic-cosine','reLU',1000,100,training_switch=training_switch,save_model=True,which_cuda=cuda)
            model4 = run(model_dir,dir,title,samples,hidden_widths,'GIN','mse','adam','cyclic-cosine','reLU',1000,100,training_switch=training_switch,save_model=True,which_cuda=cuda)
            if not os.path.exists(dir+'/GIN_'+title+'_errordensity.png'):
                evaluate(title,dir,cuda,model1,model2,model3,model4,'mse',samples)